In [1]:
import numpy as np
import pandas as pd

In [9]:
def get_df(dataset):
    return pd.read_csv(f'../Results/DirectRankerAdv/{dataset}/T-Test/Run02/results.csv')

In [10]:
law_race_results = get_df('Law-Race')
law_gender_results = get_df('Law-Gender')
compas_results = get_df('Compas')
adult_results = get_df('Adult')
results_list = [law_gender_results, law_race_results, compas_results, adult_results]

In [46]:
best_schedules = []
for i, df in enumerate(results_list):
    #median_ndcg = np.median(df['nDCG'])
    #median_rnd = np.median(df['rND'])
    schedules = np.unique(results_list[i]['Schedule'])
    for schedule in schedules:
        tmp_df = df[df['Schedule'] == schedule]
        # Get the other Schedules to compare their median to the current
        tmp_df2 = df[df['Schedule'] != schedule]
        median_ndcg = np.mean(tmp_df2['nDCG'])
        median_rnd = np.mean(tmp_df2['rND'])
        if np.median(tmp_df['rND']) < median_rnd and np.median(tmp_df['nDCG']) > median_ndcg:
            print(i, schedule)
            best_schedules.append(schedule)
print(np.unique(best_schedules, return_counts=True))

0 [1; 1]
0 [3; 1]
1 [1; 4]
1 [1; 5]
2 [1; 2]
2 [1; 3]
2 [1; 4]
2 [1; 5]
3 [1; 3]
3 [1; 4]
3 [1; 5]
(array(['[1; 1]', '[1; 2]', '[1; 3]', '[1; 4]', '[1; 5]', '[3; 1]'],
      dtype='<U6'), array([1, 1, 2, 3, 3, 1]))


In [72]:
best_schedules = []
df_names = ['Law-Gender', 'Law-Race', 'Compas', 'Adult']
for i, df in enumerate(results_list):

    schedules = np.unique(results_list[0]['Schedule'])

    df['Main Schedule'] = df.apply(lambda x: int(x['Schedule'][1]), axis=1)
    df['Adv Schedule'] = df.apply(lambda x: int(x['Schedule'][4]), axis=1)

    greater_main_df = df[df['Main Schedule'] > df['Adv Schedule']]
    greater_adv_df = df[df['Adv Schedule'] > df['Main Schedule']]
    exact_df = df[df['Adv Schedule'] == df['Main Schedule']]

    base_median_ndcg = np.median(exact_df['nDCG'])
    base_median_rnd = np.median(exact_df['rND'])
    median_greater_main_ndcg = np.median(greater_main_df['nDCG'])
    median_greater_adv_rnd = np.median(greater_adv_df['rND'])
    median_greater_main_rnd = np.median(greater_main_df['rND'])
    median_greater_adv_ndcg = np.median(greater_adv_df['nDCG'])
    #median_exact = np.median(exact_df[metric])
    print(f'{df_names[i]}')
    if median_greater_main_ndcg > base_median_ndcg:
        diff = median_greater_main_ndcg - base_median_ndcg
        print(f'Found x > y with a good nDCG Score in {df_names[i]} diffreence: {round(diff, 4)}')
    if median_greater_adv_rnd < base_median_rnd:
        diff = base_median_rnd - median_greater_adv_rnd
        print(f'Found x < y with a good rND Score in {df_names[i]} difference: {round(diff, 4)}')
    if median_greater_main_rnd <= base_median_rnd:
        diff = median_greater_main_rnd - base_median_rnd
        print(f'Found x > y with a good rND Score in {df_names[i]} diffreence: {round(diff, 4)}')
    if median_greater_adv_ndcg <= base_median_ndcg:
        diff = base_median_ndcg - median_greater_adv_ndcg
        print(f'Found x < y with a good nDCG Score in {df_names[i]} difference: {round(diff, 4)}')
    diff_rnd = round(median_greater_main_rnd - base_median_rnd, 4)
    diff_ndcg = round(median_greater_adv_ndcg - base_median_ndcg, 4)
    print(f'Difference in x > y and rND: {diff_rnd}')
    print(f'Differnece in x < y and nDCG: {diff_ndcg}')
    print(f'Correlation between x > y and nDCG: {round(df["Main Schedule"].corr(df["nDCG"]), 4)}')
    print(f'Correlation between x > y and rND: {round(df["Main Schedule"].corr(df["rND"]), 4)}')
    print(f'Correlation between x < y and nDCG: {round(df["Adv Schedule"].corr(df["nDCG"]), 4)}')
    print(f'Correlation between x < y and rND: {round(df["Adv Schedule"].corr(df["rND"]), 4)}')
    print()



Law-Gender
Difference in x > y and rND: 0.0023
Differnece in x < y and nDCG: 0.0128
Correlation between x > y and nDCG: -0.2262
Correlation between x > y and rND: 0.0208
Correlation between x < y and nDCG: 0.4111
Correlation between x < y and rND: -0.0465

Law-Race
Found x > y with a good nDCG Score in Law-Race diffreence: 0.0034
Found x < y with a good rND Score in Law-Race difference: 0.0638
Difference in x > y and rND: 0.0036
Differnece in x < y and nDCG: 0.0189
Correlation between x > y and nDCG: 0.1041
Correlation between x > y and rND: 0.4349
Correlation between x < y and nDCG: -0.3421
Correlation between x < y and rND: -0.8325

Compas
Found x > y with a good nDCG Score in Compas diffreence: 0.0153
Found x < y with a good rND Score in Compas difference: 0.1061
Found x < y with a good nDCG Score in Compas difference: 0.1212
Difference in x > y and rND: 0.0216
Differnece in x < y and nDCG: -0.1212
Correlation between x > y and nDCG: 0.5382
Correlation between x > y and rND: 0.5853
